

# End-to-End Model Management & Monitoring Demo

In this demo we cover the beginning-to-end process of:

1. Creating a deployment from a DataRobot model
2. Submitting prediction requests against the Deployment
3. Feeding back "Actuals" to the Deployment
4. Retrieving monitoring stats about the Deployment
5. Replacing the Model behind the Deployment and make new predictions against it

All of these steps are done via the API, but at any time you can view the progress in the DataRobot UI or replace one of these programmtic steps with manual clicks in the UI. 

Be sure to also follow along via the **[5-part video tutorial series](https://drive.google.com/open?id=1vohFClc1x-ieEbLELMFJewzDPqbTmdlG)**!

## Table of Contents

#### A) Setup
        1. Credentials
        2. Constants
#### B) Creating a Deployment
        1. Create Project
        2. Run Autopilot
        3. Select Model
        4. Deploy Model
        5. Configure Deployment
#### C) Submit Prediction Requests & Feed Back Actuals
        1. Prepare Predictions & Actuals Data
        2. Submit Prediction Requests
        3. Feed Back Actuals
#### D) Monitor Your Deployment
        1. Retrieve Service Stats
        2. Retrieve Accuracy Stats
#### E) Replace Your Model
        1. Select an Alternative Model
        2. Validate the Alternative Model
        3. Replace the Model
        4. Submit predictions against the replaced model


## A) Setup

In this section we need to authenticate our DataRobot API client and specify some constants that will be used throughout this procedure.

In [1]:
"""
STEP 1: Set up your credentials and instantiate the DataRobot API Client.
"""

import random
from datetime import datetime, timedelta


import datarobot as dr
import requests
import pandas as pd
import numpy as np
import os
import time

from datarobot.enums import SERVICE_STAT_METRIC

USERNAME=os.getenv('DATAROBOT_USERNAME')
DATAROBOT_API_KEY=os.getenv('DATAROBOT_API_TOKEN')
DATAROBOT_KEY=os.getenv('DATAROBOT_KEY')

# print('USERNAME:', USERNAME)
# print('DATAROBOT_API_KEY:', DATAROBOT_API_KEY)
# print('DATAROBOT_KEY:', DATAROBOT_KEY)

CLEAR_SECRETS = False  # Set to True if you'd like to clear your credentials
BASE_API_URL = 'https://app.datarobot.com/api/v2'
PREDICTION_SERVER_BASE_URL =  'https://cfds-ccm-prod.orm.datarobot.com/' # 'https://datarobot-predictions.orm.datarobot.com'

dr.Client(token=DATAROBOT_API_KEY, endpoint=BASE_API_URL)

# We will also create some functions for accessing the API directly because these routes are
# not yet exposed by the DataRobot Python SDK.
HEADERS = {
    'Content-Type': 'application/json',
    'Authorization':  'Token {}'.format(DATAROBOT_API_KEY),
}

def set_association_id(deployment_id, association_id, allow_missing_values=False):
    """Assigns the association ID for a deployment"""
    url = f'{BASE_API_URL}/deployments/{deployment_id}/settings/'
    
    data = {
        'associationId': {'requiredInPredictionRequests': not allow_missing_values, 'columnNames': [association_id]}
    }
    
    resp = requests.patch(url, json=data, headers=HEADERS)
    resp.raise_for_status()
    return


def spoof_deployment_start_date(deployment_id, start_date):
    """An internal API used for back-dating when a deployment started."""
    url = f'{BASE_API_URL}/deployments/{deployment_id}/modelHistory/currentModel/'
    
    data = {'startDate': start_date}
    
    resp = requests.patch(url, json=data, headers=HEADERS)
    resp.raise_for_status()
    return


def make_deployment_predictions(deployment_id, data, prediction_time=None):
    """
    Make predictions against a deployment.
    """
    headers = {**HEADERS, 'datarobot-key': DATAROBOT_KEY}
    if prediction_time:
        headers['X-DataRobot-Prediction-Timestamp'] = prediction_time.isoformat()

    ## Note that this uses a different base url than other API enndpoints.
    url = f'{PREDICTION_SERVER_BASE_URL}/predApi/v1.0/deployments/{deployment_id}/predictions'

    resp = requests.post(
        url,
        auth=(USERNAME, DATAROBOT_API_KEY),
        data=data,
        headers=headers
    )
    resp.raise_for_status()
    return resp.json()

USERNAME: matthew.cohen@datarobot.com
DATAROBOT_API_KEY: NWQ1NDA3ZTdmNTU1Y2QxZDQxNmZjZTYxOmFHZmZ1MlBhcUVQSHY5bzhWTjk3V05qcXBLVEpadC1R
DATAROBOT_KEY: 544ec55f-61bf-f6ee-0caf-15c7f919a45d


In [2]:
"""
STEP 2: Specify constants that will be used to configure your project, model, and deployment.
"""

# Project Info
project_name = 'MMM E2E Tutorial'
worker_count = 4

# Training Data
dataset_url = 'https://s3.amazonaws.com/datarobot-public-datasets-redistributable/10k_diabetes_20.xlsx'
target_name = 'readmitted'

# Modeling
recommendation_type = dr.enums.RECOMMENDED_MODEL_TYPE.RECOMMENDED_FOR_DEPLOYMENT

# Deployment
deployment_label = project_name
deployment_description = 'Used to practice the end-to-end process of creating and managing a deployment.'
association_id_column_name = 'event_id'

# Predictions
prediction_dataset_url = 'https://s3.amazonaws.com/datarobot_public_datasets/exploratory_testing/mmm_accuracy/10k_predictions_alphatesting_associd.csv'
prediction_sample_size = 1000  # The number of rows to randomly sample from the above files and submit for predictions.
past_n_days = 365  # We will spoof when our predictions were submitted, going back this many days until today.
num_prediction_chunks = 100  # We will spread out our predictions into this many chunks across the above time range.

# Actuals
date_format = '%A %-d, %Y'
actuals_dataset_url = 'https://s3.amazonaws.com/datarobot_public_datasets/exploratory_testing/mmm_accuracy/10k_actuals_alphatesting.csv'

# Model Monitoring
monitoring_window_start=(datetime.now() - timedelta(days=past_n_days)).replace(hour=0, minute=0, second=0, microsecond=0)
monitoring_window_end=datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

# Model Replacement
model_replacement_recommendation_type = dr.enums.RECOMMENDED_MODEL_TYPE.MOST_ACCURATE
model_replacement_reason = dr.enums.MODEL_REPLACEMENT_REASON.ACCURACY


## B) Creating a Deployment

Here we create a new DataRobot project, run Autopilot, select a recommended model, create a Deployment from it, and
set crucial Deployment settings.

In [3]:
"""
STEP 1: Find an existing project by the specifed name or create one if needed.
"""
projects = dr.Project.list(search_params={'project_name': project_name})

if not projects:
    project = dr.Project.create(dataset_url, project_name)
else:
    project = projects[0]

print(f'Using project "{project.project_name}" with ID {project.id}')

Using project "MMM E2E Tutorial" with ID 5e920d5590f0540f2418712a


In [4]:
"""
STEP 2: Set our target, kick off autopilot, and wait for it to complete.
"""
try:
    project.set_target(
        target=target_name,
        mode=dr.AUTOPILOT_MODE.FULL_AUTO,
        worker_count=worker_count,
    )
except dr.errors.ClientError as err:
    if err.status_code == 422:
        print(f'Target has already been set to `{target_name}`.')
    else:
        raise
else:
    print(f'The target has been set to `{target_name}`')
    project.wait_for_autopilot(verbosity=dr.VERBOSITY_LEVEL.SILENT)

print('Autopilot is complete and we are ready to select a model to deploy.')

Target has already been set to `readmitted`.
Autopilot is complete and we are ready to select a model to deploy.


In [5]:
"""
STEP 3: Identify a model to deploy
"""
use_recommended_model = True
fallback_model_id = None

if use_recommended_model:
    recommended_model = dr.ModelRecommendation.get(project.id, recommendation_type)
    model = dr.Model.get(recommended_model.project_id, recommended_model.model_id)
    print(f'DataRobot recommends a {model.model_type} model with ID {model.id}.')
elif fallback_model_id:
    model = dr.Model.get(project.id, fallback_model_id)
else:
    raise RuntimeError('You must specify a fallback_model_id if you choose not to use a recommended model.')

print(f'Using a {model.model_type} model with ID {model.id}.')

DataRobot recommends a RandomForest Classifier (Gini) model with ID 5e9210b0e5620f181e6aeb2a.
Using a RandomForest Classifier (Gini) model with ID 5e9210b0e5620f181e6aeb2a.


In [6]:
"""
STEP 4: Create a Deployment for this project/model
"""
existing_deployments = dr.Deployment.list()
deployment = None
for existing_deployment in existing_deployments:
    if (
        existing_deployment.model and
        existing_deployment.model['id'] == model.id and
        existing_deployment.model['project_id'] == project.id
    ):
        deployment = existing_deployment
        break

if not deployment:
    print("No deployment exists for this project/model yet. Creating one now.")
    prediction_server = dr.PredictionServer.list()[0]
    deployment = dr.Deployment.create_from_learning_model(
        model.id,
        label=deployment_label,
        description=deployment_description,
        default_prediction_server_id=prediction_server.id
    )

print(f'Using Deployment with label "{deployment.label}" and ID {deployment.id}')

No deployment exists for this project/model yet. Creating one now.
Using Deployment with label "MMM E2E Tutorial" and ID 5e966280ca2c130201246917


In [7]:
"""
STEP 5: Assign an association ID to the deployment and set up drift tracking
"""
print('Enable drift tracking')
t0 = time.time()
deployment.update_drift_tracking_settings(target_drift_enabled=True, feature_drift_enabled=True)
print('- time: %0.2f' % (time.time() - t0))

t0 = time.time()
print('Set association id to column "%s"' % association_id_column_name)
deployment.update_association_id_settings(column_names=[association_id_column_name], required_in_prediction_requests=True)
print('- time: %0.2f' % (time.time() - t0))

print('Your Deployment is set up to use Drift Tracking and capture Association IDs! Now you can monitor it effectively.')

Enable drift tracking
- time: 43.17
Set association id to column "event_id"
- time: 1.19
Your Deployment is set up to use Drift Tracking and capture Association IDs! Now you can monitor it effectively.



## C) Submit Prediction Requests & Feed Back Actuals

Now we can actually start making prediction requests against our deployment. We spoof the time of our predictions
so that we can simulate predictions made over the course of many days.

We also use this section to submit "actuals" – data about what the real-world outcome was for each prediction request, identifiable by our previously set Association ID.

In [8]:
"""
STEP 1: Create a random sample of prediction & corresponding actual data for use in future steps.

We do this simply because we might not want to send up all of our prediction data in one go,
but instead, just a subset of it.
"""

prediction_dataset = pd.read_csv(prediction_dataset_url)
prediction_dataset = prediction_dataset.sample(n=prediction_sample_size)

actuals_dataset = pd.read_csv(actuals_dataset_url)

merged_inner = pd.merge(left=actuals_dataset,right=prediction_dataset, left_on='associationId', right_on=association_id_column_name)
actuals_dataset = merged_inner[['associationId', 'actualValue', 'wasActedOn']]
actuals_dataset = actuals_dataset.rename(columns={'associationId': 'association_id', 'actualValue': 'actual_value', 'wasActedOn': 'was_acted_on'})

print(f'Prepared {len(prediction_dataset.index)} predictions rows and {len(actuals_dataset.index)} corresponding actuals.')

Prepared 1000 predictions rows and 1000 corresponding actuals.


In [9]:
"""
STEP 2: Make some predictions, spoofed to be across a time range.

We want to chunk up our predictions and spoof the time that we submitted each prediction. This allows us to
create more realistic demo data.

We do this by:
  1) Selecting a random subset of size x from all prediction data available
  2) Selecting n random days over the past m days
  3) Splitting the x predictions roughly equally across the n days and submitting prediction requests for each of those days
"""

def chunk(a, n):
    """
    Chunks a list up into a list of lists, with each inner list of roughly equal lengths.
    Algorithm source: https://stackoverflow.com/a/2135920
    """
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))
        
def generate_random_days_in_past(number_of_days, max_days_to_go_back):
    """Generates a list of unique random integers, each representing a date n days before now."""
    random_days_in_past = random.sample(range(1, max_days_to_go_back), number_of_days)
    random_days_in_past.sort(reverse=True)
    random_days_in_past = [datetime.now() - timedelta(days=n_days_ago) for n_days_ago in random_days_in_past]
    return random_days_in_past


deployment_start_date = (datetime.now() - timedelta(days=past_n_days)).isoformat()
print(f'About to trick the deployment into thinking it was created on {deployment_start_date} so that all prediction show up in the UI.\n')
spoof_deployment_start_date(deployment.id, deployment_start_date)

print(f'About to submit {len(prediction_dataset.index)} predictions across {num_prediction_chunks} chunks over the past {past_n_days} days.\n')

random_days_in_past = generate_random_days_in_past(num_prediction_chunks, past_n_days)
for index, prediction_chunk in enumerate(chunk(prediction_dataset, num_prediction_chunks)):
    predictions_data = prediction_chunk.to_json(orient='records')
    
    spoofed_time_of_prediction_chunk = random_days_in_past[index]
    make_deployment_predictions(deployment.id, predictions_data, prediction_time=spoofed_time_of_prediction_chunk)
    print(f'Submitted {len(prediction_chunk.index)} predictions, spoofed to have a timestamp of {spoofed_time_of_prediction_chunk.isoformat()}')

print(f'\nCompleted making {len(prediction_dataset.index)} predictions across {num_prediction_chunks} chunks in the past {past_n_days} days.')

About to trick the deployment into thinking it was created on 2019-04-15T18:26:10.014646 so that all prediction show up in the UI.

<Response [202]>
About to submit 1000 predictions across 100 chunks over the past 365 days.

Submitted 10 predictions, spoofed to have a timestamp of 2019-04-18T18:26:10.712134
Submitted 10 predictions, spoofed to have a timestamp of 2019-04-20T18:26:10.712145
Submitted 10 predictions, spoofed to have a timestamp of 2019-04-22T18:26:10.712147
Submitted 10 predictions, spoofed to have a timestamp of 2019-04-24T18:26:10.712149
Submitted 10 predictions, spoofed to have a timestamp of 2019-04-26T18:26:10.712150
Submitted 10 predictions, spoofed to have a timestamp of 2019-04-27T18:26:10.712152
Submitted 10 predictions, spoofed to have a timestamp of 2019-05-01T18:26:10.712154
Submitted 10 predictions, spoofed to have a timestamp of 2019-05-03T18:26:10.712155
Submitted 10 predictions, spoofed to have a timestamp of 2019-05-05T18:26:10.712157
Submitted 10 predic

Submitted 10 predictions, spoofed to have a timestamp of 2020-03-15T18:26:10.712296
Submitted 10 predictions, spoofed to have a timestamp of 2020-03-17T18:26:10.712298
Submitted 10 predictions, spoofed to have a timestamp of 2020-03-18T18:26:10.712299
Submitted 10 predictions, spoofed to have a timestamp of 2020-03-20T18:26:10.712301
Submitted 10 predictions, spoofed to have a timestamp of 2020-03-23T18:26:10.712303

Completed making 1000 predictions across 100 chunks in the past 365 days.


In [13]:
"""
STEP 3: Feed back actuals to the deployment
"""

actuals_data = actuals_dataset.to_dict(orient='records')

# Submit the actuals via the API
deployment.submit_actuals(actuals_data)

print(f'Completed submitting {len(actuals_data)} actuals.')


Completed submitting 1000 actuals.



## D) Monitor Your Deployment


In [14]:
"""
STEP 1: Get summarized Service Stats about our Deployment through a configurable window of time.
"""
print(SERVICE_STAT_METRIC.ALL)
print('monitoring_window_start:', monitoring_window_start)
# print('monitoring_window_end:', monitoring_window_end)
# service_stats = deployment.get_service_stats(
#     start_time=monitoring_window_start,
#     end_time=monitoring_window_end,
# )
print('monitoring_window_end:', monitoring_window_end)
service_stats = deployment.get_service_stats(
    start_time=datetime(2019, 8, 1, hour=15),
    end_time=datetime(2019, 8, 8, hour=15)
)
# service_stats = deployment.get_service_stats()

print(f'Deployment stats from {monitoring_window_start.strftime(date_format)} to {monitoring_window_end.strftime(date_format)}')
print('----------------------------------------------------------')
for stat, value in service_stats.metrics.items():
    print(f'{stat}: {value}')

# NOTE: You may also be interested in getting these stats over time, which you can
#       do with deployment.get_service_stats_over_time

['totalPredictions', 'totalRequests', 'slowRequests', 'executionTime', 'responseTime', 'userErrorRate', 'serverErrorRate', 'numConsumers', 'cacheHitRatio', 'medianLoad', 'peakLoad']
monitoring_window_start: 2019-04-15 00:00:00
monitoring_window_end: 2020-04-14 00:00:00
Deployment stats from Monday 15, 2019 to Tuesday 14, 2020
----------------------------------------------------------
totalPredictions: 20
userErrorRate: 0
cacheHitRatio: 1
executionTime: 21
totalRequests: 2
serverErrorRate: 0
slowRequests: 0
medianLoad: 0
numConsumers: 1
responseTime: 209
peakLoad: 1


In [15]:
"""
STEP 2: Get summarized Accuracy Stats about our Deployment through a configurable window of time.
"""

accuracy = deployment.get_accuracy(
    start=monitoring_window_start,
    end=monitoring_window_end
)

print(f'Accuracy stats from {monitoring_window_start.strftime(date_format)} to {monitoring_window_end.strftime(date_format)}')
print('----------------------------------------------------------')
for metric, values in accuracy.metrics.items():
    print(f'{metric}:')
    print(f'    Baseline: {values["baseline_value"]}')
    print(f'    Value: {values["value"]}')
    print(f'    % Change: {values["percent_change"]}')
          
# NOTE: You may also be interested in getting these stats over time, which you can
#       do with deployment.get_accuracy_over_time

Accuracy stats from Monday 15, 2019 to Tuesday 14, 2020
----------------------------------------------------------
LogLoss:
    Baseline: 0.610058956353653
    Value: 0.5848363513983
    % Change: 4.13
AUC:
    Baseline: 0.7026362590229103
    Value: 0.7516499999999999
    % Change: 6.98
Kolmogorov-Smirnov:
    Baseline: 0.30269902709488444
    Value: 0.3708333333333333
    % Change: 22.51
Rate@Top10%:
    Baseline: 0.8048780487804879
    Value: 0.7722772277227723
    % Change: -4.05
Gini Norm:
    Baseline: 0.40527251804582054
    Value: 0.5032999999999999
    % Change: 24.19



## E) Replace Your Model

Here we find another model from our original Autopilot run that DataRobot recommends for a different reason,
validate that we can replace our Deployment's current model with this new one, and then perform the replacement.

Finally, we perform additional predictions against our deployment without even having to reference a model, thereby
proving that model replacement is a seemless process thanks to the concept of Deployments.

In [16]:
"""
STEP 1: Find an alternative model from Autopilot that DataRobot recommended for another reason.
"""

# First we try to get a model that was recommended for another reason.
recommended_new_model = dr.ModelRecommendation.get(project.id, model_replacement_recommendation_type)
replacement_model = dr.Model.get(recommended_model.project_id, recommended_model.model_id)

# But sometimes, it is the same as the model that we started with. In this case, just
# pick another model of a different type.
if replacement_model.id == model.id:
    models = project.get_models()
    for new_model in models:
        if new_model.id != model.id and new_model.model_type != model.model_type:
            replacement_model = new_model
            break

print(f"Selected a {replacement_model.model_type} model with ID {replacement_model.id} to use as our Deployment's new active model.")

Selected a Gradient Boosted Trees Classifier model with ID 5e920f2ee5620f146d6aeb12 to use as our Deployment's new active model.


In [17]:
"""
STEP 2: Validate that this model can be used to replace our Deployment's current model.
"""

status, message, checks = deployment.validate_replacement_model(new_model_id=replacement_model.id)

print(f'Status: {status}; {message}')

Status: passing; Model can be used to replace the current model of the deployment.


In [18]:
"""
STEP 3: Actually perform the model replacement.
"""

deployment.replace_model(replacement_model.id, model_replacement_reason)

print(f"The Deployment is now using a {deployment.model['type']} model with ID.")

The Deployment is now using a Gradient Boosted Trees Classifier model with ID.


In [19]:
"""
STEP 4: Submit new predictions against the replaced model.

Notice that despite having a new model deployed, the call to make predictions _does not change_ from
our previous one because we reference the deployment NOT the model. This illustrates how easy it is
to swap models without having to change application code.
"""

new_prediction_dataset = prediction_dataset.sample(n=10)
make_deployment_predictions(deployment.id, new_prediction_dataset.to_json(orient='records'), prediction_time=spoofed_time_of_prediction_chunk)

print(f'Completed making {len(new_prediction_dataset.index)} new predictions against the deployment\'s new model.')

Completed making 10 new predictions against the deployment's new model.
